<a href="https://colab.research.google.com/github/esnue/ThesisAllocationSystem/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

As part of the workflow between GitHub and Google Colab, please follow these steps: 
1. Upload the data to a folder in your GDrive. 
2. Mount your GDrive.
3. Set the data folder as your present working directory. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd
%cd /content/drive/MyDrive/ThesisAllocationSystem

# Convert PDF to TXT

Convert all PDF files in the current working directory to TXT files.

In [ ]:
!pip install tika

In [31]:
import os
from tika import parser

def read_pdf(pdf_file):

    text = parser.from_file(pdf_file)['content']
    return text.encode("latin-1","ignore")

def pdf_to_txt(folder_with_pdf, dest_folder):
    pdf_files = []

    for root, dirs, files in os.walk(folder_with_pdf):
        for f in files:
            if '.pdf' in f:
                pdf_files.append(os.path.join(root, f))
    #print(pdf_files)

    for file_ in pdf_files:
        text_file = os.path.splitext(os.path.basename(file_))[0]+'.txt'
        with open(os.path.join(dest_folder,text_file), 'wb') as text_f:
            text_f.write(read_pdf(file_))

    return None

In [ ]:
pdf_to_txt('./supervisors', './supervisors-txt') 

In [ ]:
pdf_to_txt('./train-papers', './train-papers-txt') 

In [ ]:
pdf_to_txt('./raw dat', './output') 

In [ ]:
pdf_to_txt('./raw dat', './output') 

In [ ]:
pdf_to_txt('./raw dat', './output') 

# Put TXT files into CSV

After importing the packages, define the directory of interest and run the function below to create a CSV files that entails all TXT files in the following structure: 

Filename | Content 

In [32]:
import csv
from pathlib import Path

In [58]:
def txt_to_csv(x): 

    os.chdir('/content/drive/MyDrive/ThesisAllocationSystem/' + x)

    with open(x + '.csv', 'w', encoding = 'Latin-1') as out_file:
        csv_out = csv.writer(out_file)
        csv_out.writerow(['FileName', 'Content'])
        for fileName in Path('.').glob('*.txt'):
            lines = [ ]
            with open(str(fileName.absolute()),'rb') as one_text:
                for line in one_text.readlines():
                    lines.append(line.decode(encoding='Latin-1',errors='ignore').strip())
            csv_out.writerow([str(fileName),' '.join(lines)])

In [ ]:
txt_to_csv('train-papers-txt')

In [ ]:
txt_to_csv('test-theses')

In [ ]:
txt_to_csv('test-proposals')

In [12]:
txt_to_csv('supervisors')